<a href="https://www.kaggle.com/code/vtrackstar/health-and-wellness-adherence-reporting?scriptVersionId=251570712" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# How to make reporting scalable for business
# clients = {
#    'Marvel': '1t2eBcYGvtNocz_-20akb0fsZPuRscWFr6l0yjQuyYLw',
#    'Tony': 'YOUR_SECOND_CLIENT_SHEET_ID',
    # Add more clients here
# }

# for client_name, sheet_id in clients.items():
#    print(f"\nBuilding report for {client_name.capitalize()}...")
    
#    gid = '1380274994'  # Stays the same if same Google sheet template is used for each client
#    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={gid}"
    
#    df = pd.read_csv(url)

In [3]:
sheet_name = 'Food_Diary_Marvel'
sheet_id = '1t2eBcYGvtNocz_-20akb0fsZPuRscWFr6l0yjQuyYLw'
url = "https://docs.google.com/spreadsheets/d/1t2eBcYGvtNocz_-20akb0fsZPuRscWFr6l0yjQuyYLw/gviz/tq?tqx=out:csv&gid=1380274994"

In [4]:
df = pd.read_csv(url)
df
# Warning below can be ignored as that is caused by NaN values

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Daily Totals,Calories (kcal),Fat (g),Carbs (g),Protein (g),Calories Burned (Apple Watch),Calories Burned (WHOOP),BMR,Caloric Deficit (Apple Watch),Caloric Deficit (WHOOP),Weight (lbs.),Unnamed: 11,Diet Adherence Predictive Model External Link,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,3-Mar-2025,3457.519736,172.62,219.30,277.64,1552.0,NaN,2435.6,530.1,-1021.9,260.0,NaN,This is the totals page. The totals are used m...,NaN,NaN,NaN,NaN,NaN
1,4-Mar-2025,2515.878522,119.78,145.19,224.05,2419.0,NaN,2435.6,2338.7,-80.3,260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5-Mar-2025,3096.439487,144.13,164.59,295.72,903.0,NaN,2435.6,242.1,-660.9,260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6-Mar-2025,2428.879383,121.31,80.43,258.70,526.0,NaN,2435.6,532.7,6.7,260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7-Mar-2025,2358.335948,89.93,132.90,254.58,1491.0,NaN,2435.6,1568.2,77.2,260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2-Nov-2025,0.000000,-,-,-,NaN,NaN,855.6,855.6,855.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,3-Nov-2025,0.000000,-,-,-,NaN,NaN,855.6,855.6,855.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,4-Nov-2025,0.000000,-,-,-,NaN,NaN,855.6,855.6,855.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,5-Nov-2025,0.000000,-,-,-,NaN,NaN,855.6,855.6,855.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df[df.columns[:5]].head()

,Daily Totals,Calories (kcal),Fat (g),Carbs (g),Protein (g)
0,3-Mar-2025,3457.519736,172.62,219.30,277.64
1,4-Mar-2025,2515.878522,119.78,145.19,224.05
2,5-Mar-2025,3096.439487,144.13,164.59,295.72
3,6-Mar-2025,2428.879383,121.31,80.43,258.70
4,7-Mar-2025,2358.335948,89.93,132.90,254.58


In [6]:
df.columns

Index(['Daily Totals', 'Calories (kcal)', 'Fat (g)', 'Carbs (g)',
       'Protein (g)', 'Calories Burned (Apple Watch)',
       'Calories Burned (WHOOP)', 'BMR', 'Caloric Deficit (Apple Watch)',
       'Caloric Deficit (WHOOP)', 'Weight (lbs.)', 'Unnamed: 11',
       'Diet Adherence Predictive Model External Link', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
      dtype='object')

In [7]:
df_clean = df[['Daily Totals', 'Calories (kcal)', 'Fat (g)', 'Carbs (g)',
       'Protein (g)']]
df_a = df_clean.dropna()
df_a

,Daily Totals,Calories (kcal),Fat (g),Carbs (g),Protein (g)
0,3-Mar-2025,3457.519736,172.62,219.30,277.64
1,4-Mar-2025,2515.878522,119.78,145.19,224.05
2,5-Mar-2025,3096.439487,144.13,164.59,295.72
3,6-Mar-2025,2428.879383,121.31,80.43,258.70
4,7-Mar-2025,2358.335948,89.93,132.90,254.58
...,...,...,...,...,...
243,1-Nov-2025,0.000000,-,-,-
244,2-Nov-2025,0.000000,-,-,-
245,3-Nov-2025,0.000000,-,-,-
246,4-Nov-2025,0.000000,-,-,-


In [8]:
print(df_a.isnull().sum())
print(df_a.info())

Daily Totals       0
Calories (kcal)    0
Fat (g)            0
Carbs (g)          0
Protein (g)        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 248 entries, 0 to 247
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Daily Totals     248 non-null    object 
 1   Calories (kcal)  248 non-null    float64
 2   Fat (g)          248 non-null    object 
 3   Carbs (g)        248 non-null    object 
 4   Protein (g)      248 non-null    object 
dtypes: float64(1), object(4)
memory usage: 11.6+ KB
None


In [9]:
df_c = df_a.copy()
df_c['Daily Totals'] = pd.to_datetime(df_c['Daily Totals'], errors='coerce', dayfirst=True)
df_c['Calories (kcal)'] = pd.to_numeric(df_c['Calories (kcal)'], errors='coerce')
df_c['Protein (g)'] = pd.to_numeric(df_c['Protein (g)'], errors='coerce')
df_c['Fat (g)'] = pd.to_numeric(df_c['Fat (g)'], errors='coerce')
df_c['Carbs (g)'] = pd.to_numeric(df_c['Carbs (g)'], errors='coerce')

print(df_c.info())

<class 'pandas.core.frame.DataFrame'>
Index: 248 entries, 0 to 247
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Daily Totals     248 non-null    datetime64[ns]
 1   Calories (kcal)  248 non-null    float64       
 2   Fat (g)          21 non-null     float64       
 3   Carbs (g)        21 non-null     float64       
 4   Protein (g)      21 non-null     float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 11.6 KB
None


In [10]:
BMR = 66.47 + (6.24*257) + (12.7*78) - (6.76*28) # BMR MSJ formula in lb & in
TDEE = BMR*1.55 # moderately active male activity level multiplier

calorie_goals = {
    'super_cut': TDEE - 750,
    'cut': TDEE - 500,
    'maintain': TDEE,
    'bulk': TDEE + 500,
    'fast_bulk': TDEE + 750
}

tdee_df = pd.DataFrame([calorie_goals])
tdee_df.insert(0, 'BMR', BMR)
tdee_df.insert(1, 'TDEE', TDEE)

print("BMR, TDEE, Daily Caloric Need Based on Goal")
tdee_df

BMR, TDEE, Daily Caloric Need Based on Goal


,BMR,TDEE,super_cut,cut,maintain,bulk,fast_bulk
0,2471.47,3830.7785,3080.7785,3330.7785,3830.7785,4330.7785,4580.7785


In [11]:
df1 = df_c.copy()
df1['calories_needed_maintainence'] = df1['Calories (kcal)'] - calorie_goals['maintain']
df1['calories_needed_bulk'] = df1['Calories (kcal)'] - calorie_goals['bulk']
df1['calories_needed_super_cut'] = df1['Calories (kcal)'] - calorie_goals['super_cut']
df1['protein_in_grams_needed'] = (0.8*257) - df1['Protein (g)']
print("Daily Caloric and Protein Surplus/Deficit Based On Goal")
df1.head()

Daily Caloric and Protein Surplus/Deficit Based On Goal


,Daily Totals,Calories (kcal),Fat (g),Carbs (g),Protein (g),calories_needed_maintainence,calories_needed_bulk,calories_needed_super_cut,protein_in_grams_needed
0,2025-03-03,3457.519736,172.62,219.30,277.64,-373.258764,-873.258764,376.741236,-72.04
1,2025-03-04,2515.878522,119.78,145.19,224.05,-1314.899978,-1814.899978,-564.899978,-18.45
2,2025-03-05,3096.439487,144.13,164.59,295.72,-734.339013,-1234.339013,15.660987,-90.12
3,2025-03-06,2428.879383,121.31,80.43,258.70,-1401.899117,-1901.899117,-651.899117,-53.10
4,2025-03-07,2358.335948,89.93,132.90,254.58,-1472.442552,-1972.442552,-722.442552,-48.98


In [12]:
dfa = df1.copy()
dfa['adherence'] = ((dfa['Calories (kcal)'] <= calorie_goals['cut']) & (dfa['protein_in_grams_needed'] < 0)).astype(int)
dfa[['Daily Totals', 'Calories (kcal)', 'adherence', 'protein_in_grams_needed']].head()
# Warning below can be ignored as that is caused by NaN values

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


,Daily Totals,Calories (kcal),adherence,protein_in_grams_needed
0,2025-03-03,3457.519736,0,-72.04
1,2025-03-04,2515.878522,1,-18.45
2,2025-03-05,3096.439487,1,-90.12
3,2025-03-06,2428.879383,1,-53.10
4,2025-03-07,2358.335948,1,-48.98


In [13]:
n1df = tdee_df.copy()
n1df

,BMR,TDEE,super_cut,cut,maintain,bulk,fast_bulk
0,2471.47,3830.7785,3080.7785,3330.7785,3830.7785,4330.7785,4580.7785


In [14]:
newdf = dfa.copy()
newdf = newdf.rename(columns={
    'calories_needed_maintainence': 'Calories (kCal) Needed To Maintain',
    'calories_needed_bulk': 'Calories (kCal) Needed To Bulk',
    'calories_needed_super_cut': 'Calories (kCal) Needed For Super Cut',
    'protein_in_grams_needed': 'Additional Protein Needed (g)'
})
newdf["Adherence to 'Cut' Goal"] = newdf['adherence'].map({1: 'On Track', 0: 'Off Track'})
print(newdf.columns)

Index(['Daily Totals', 'Calories (kcal)', 'Fat (g)', 'Carbs (g)',
       'Protein (g)', 'Calories (kCal) Needed To Maintain',
       'Calories (kCal) Needed To Bulk',
       'Calories (kCal) Needed For Super Cut', 'Additional Protein Needed (g)',
       'adherence', 'Adherence to 'Cut' Goal'],
      dtype='object')


In [15]:
recent_30 = newdf.sort_values('Daily Totals', ascending=False)
recent_30 = recent_30.sort_values('Daily Totals')

# Get today's date without time and filter to only keep on or before the date the report is ran
today = pd.Timestamp.today().normalize()
# specific_date = pd.Timestamp("2025-07-20")  Format: "YYYY-MM-DD"
# Use 'specific_date' instead of 'today' if you want the report ran from a certain date instead of today
recent30 = recent_30[recent_30['Daily Totals'] <= today]
n2df1 = recent30[['Daily Totals', 'Calories (kcal)', 'Fat (g)', 'Carbs (g)',
       'Protein (g)', 'Additional Protein Needed (g)',
       "Adherence to 'Cut' Goal"]].copy()
n2df = n2df1.sort_values('Daily Totals', ascending=False).head(30)
n2df

# Warning below can be ignored as that is caused by NaN values

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Daily Totals,Calories (kcal),Fat (g),Carbs (g),Protein (g),Additional Protein Needed (g),Adherence to 'Cut' Goal
139,2025-07-20,0.0,NaN,NaN,NaN,NaN,Off Track
138,2025-07-19,0.0,NaN,NaN,NaN,NaN,Off Track
137,2025-07-18,0.0,NaN,NaN,NaN,NaN,Off Track
136,2025-07-17,0.0,NaN,NaN,NaN,NaN,Off Track
135,2025-07-16,0.0,NaN,NaN,NaN,NaN,Off Track
134,2025-07-15,0.0,NaN,NaN,NaN,NaN,Off Track
133,2025-07-14,0.0,NaN,NaN,NaN,NaN,Off Track
132,2025-07-13,0.0,NaN,NaN,NaN,NaN,Off Track
131,2025-07-12,0.0,NaN,NaN,NaN,NaN,Off Track
130,2025-07-11,0.0,NaN,NaN,NaN,NaN,Off Track


In [16]:
newdf1 = newdf.dropna()
newdf2 = newdf1.copy()
adherence_summary = newdf2['adherence'].value_counts().rename_axis('adherence_status').reset_index(name='count')
adherence_summary['Adherence Status'] = adherence_summary['adherence_status'].map({1: 'On Track', 0: 'Off Track'})

n3df = adherence_summary[['Adherence Status', 'count']].copy()

print("\n🔹 Diet (Goal: Cut) Adherence Summary Report 🔹\n")
n3df


🔹 Diet (Goal: Cut) Adherence Summary Report 🔹



,Adherence Status,count
0,On Track,11
1,Off Track,10


In [17]:
# Automated PDF Report Builder
# Can be run weekly (for coaches) or monthly  (for clients/leadership) as a progress report

# Create a PDF
with PdfPages("fitness_report.pdf") as pdf:
    
    # Function to plot a table and save to PDF
    def add_df_to_pdf(df, title, pdf):
        fig, ax = plt.subplots(figsize=(10, len(df)*0.5 + 1))  # Dynamically adjust height
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
        table.scale(1, 1.5)
        plt.title(title, fontsize=14, pad=20)
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
    
    # Add each DataFrame with a title
    add_df_to_pdf(n1df, "BMR And TDEE Based on Goal for Client: Marvel J.", pdf)
    add_df_to_pdf(n2df, "Daily Totals (Last 30 Entries) with Goal Adherence Tracker", pdf)
    add_df_to_pdf(n3df, "Diet (Goal: Cut) Adherence Summary Report for Completed Entries", pdf)

print("✅ Report saved as 'fitness_report.pdf'")

✅ Report saved as 'fitness_report.pdf'


In [18]:
# Scalable method to save report PDF by client
# output_pdf = f"reports/{client_name}_fitness_report.pdf"

# with PdfPages(output_pdf) as pdf:
    
#    def add_df_to_pdf(df, title, pdf):
#        fig, ax = plt.subplots(figsize=(10, len(df)*0.5 + 1))  # Adjust height dynamically
#        ax.axis('tight')
#        ax.axis('off')
#        table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
#        table.scale(1, 1.5)
#        plt.title(title, fontsize=14, pad=20)
#        pdf.savefig(fig, bbox_inches='tight')
#        plt.close()
    
    # Use dynamic client name in report title
#    add_df_to_pdf(n1df, f"BMR And TDEE Based on Goal for Client: {client_name}", pdf)
#    add_df_to_pdf(n2df, "Daily Totals (Last 30 Entries) with Goal Adherence Tracker", pdf)
#    add_df_to_pdf(n3df, f"Diet (Goal: Cut) Adherence Summary Report for Completed Entries", pdf)

# print(f"✅ Report saved as '{output_pdf}'")